In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import random
import os
import pretty_midi
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adamax
from pydub import AudioSegment

# Set random seed for reproducibility
random.seed(42)
np.random.seed(42)
tf.random.set_seed(42)

In [2]:
def load_midi_files(directory):
    notes = []
    for filename in os.listdir(directory):
        if filename.endswith(".mid"):
            midi_data = pretty_midi.PrettyMIDI(os.path.join(directory, filename))
            for instrument in midi_data.instruments:
                if instrument.is_drum:
                    continue
                for note in instrument.notes:
                    pitch = note.pitch
                    duration = note.end - note.start
                    notes.append((pitch, duration))
    return notes


In [3]:
data_directory = "./dataset/"
notes = load_midi_files(data_directory)
pitchnames = sorted(set(note[0] for note in notes))
note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

C:\Users\adith\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\pretty_midi\pretty_midi.py:100: RuntimeWarning: Tempo, Key or Time signature change events found on non-zero tracks.  This is not a valid type 0 or type 1 MIDI file.  Tempo, Key or Time Signature may be wrong.
  warnings.warn(
